In [50]:
import pandas as pd
import pickle
import warnings

%run models.ipynb
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

warnings.filterwarnings("ignore")

In [92]:
dataset = "tacred"

features_with_labels = pd.read_csv(f"/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Features/Latest/features_{dataset}.csv",
                                   index_col=0)

# features_with_labels["label"] = features_with_labels.apply(lambda x : 1 if x["rel type"] == "no_relation" else x["label"], axis = 1)
## Sampling 20% of data
# features_with_labels_sampled = features_with_labels.sample(frac=0.2, random_state=100)
#, 'entity_tag_1', 'entity_tag_2'
columns = ['sentence_length', 'root_word', 'root_index',
       'entity_1_index', 'entity_2_index', 'entity_distance',
       'no_words_before_entity_1', 'no_words_after_entity_2',
       'entity_1_root_distance', 'entity_2_root_distance', 'entity 1 name',
       'entity 2 name', 'entity_type_1', 'entity_type_2', 'entity_pos_1',
       'entity_pos_2', 'entity_dep_1', 'entity_dep_2', 'shortest_distance', 'label']
features_with_labels = features_with_labels[columns]
display(features_with_labels.groupby(by=["label"]).count()["root_word"])

df_pos = features_with_labels[features_with_labels["label"]==1]
df_neg = features_with_labels[features_with_labels["label"]==0].sample(len(df_pos))
df = df_pos.append(df_neg)
display(df)

## Sampling 
# features_with_labels = features_with_labels.sample(frac = 0.1, random_state=101)
# features_with_labels = features_with_labels.sample(frac = 0.3,  
                                                   # weights = features_with_labels.groupby('label')['label'].transform('count'))

X = df.iloc[:,:-1]
y =  df.iloc[:,-1]
error_rows = X[X.isna().any(axis=1)]
X = X.fillna("")

x_train_original, x_test_original, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

onehot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
x_train = onehot_encoder.fit_transform(x_train_original)
x_test = onehot_encoder.transform(x_test_original)

features = onehot_encoder.get_feature_names(input_features=df.columns[:-1])
display(len(x_train), len(x_test))


label
0    17440
1    2960 
Name: root_word, dtype: int64

,sentence_length,root_word,root_index,entity_1_index,entity_2_index,entity_distance,no_words_before_entity_1,no_words_after_entity_2,entity_1_root_distance,entity_2_root_distance,entity 1 name,entity 2 name,entity_type_1,entity_type_2,entity_pos_1,entity_pos_2,entity_dep_1,entity_dep_2,shortest_distance,label
3,18,was,3,1,7,6,0,10,2,4,Dunne,John Gregory Dunne,PER,PER,PROPN,PROPN-PROPN-PROPN,poss,compound-compound-appos,6,1
4,18,was,3,1,7,6,0,10,2,4,Dunne,John Gregory Dunne,PER,PER,PROPN,PROPN-PROPN-PROPN,poss,compound-compound-appos,6,1
5,18,was,3,1,7,6,0,10,2,4,Dunne,John Gregory Dunne,PER,PER,PROPN,PROPN-PROPN-PROPN,poss,compound-compound-appos,6,1
10,28,include,3,1,20,19,0,7,2,17,Mays,Billy Mays III,PER,PER,NOUN,PROPN-PROPN-PROPN,poss,compound-compound-appos,8,1
11,47,emerged,2,11,17,6,10,29,-9,15,Kissel,Robert,PER,PER,PROPN,PROPN,nsubj,poss,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7546,25,showed,2,20,22,2,19,2,-18,20,Meredith,Ellen Pompeo,PER,PER,PROPN,PROPN-PROPN,npadvmod,compound-ROOT,3,0
3228,28,helped,10,8,22,14,7,5,2,12,North,Hwang,LOC,PER,PROPN,PROPN,pobj,nsubj,2,0
14235,16,coming,9,4,7,3,3,8,5,-2,Alexandra Burke,Jade Ewen,PER,PER,PROPN-PROPN,PROPN-PROPN,compound-nsubj,compound-conj,1,0
1582,21,become,6,4,11,7,3,9,2,5,Douglas Flint,Stephen Green,PER,PER,PROPN-PROPN,PROPN-PROPN,compound-nsubj,compound-dobj,3,0


5328

592

In [93]:
pred = logistic_regression(x_train, x_test, y_train, y_test, "binary")
x_test_log_pred = x_test_original
x_test_log_pred["label"] = y_test
x_test_log_pred["prediction"] = pred
x_test_log_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_logisitic_regression_prediction.csv")


--------Logistic Regression-----------


In [94]:
pred = random_forest(x_train, x_test, y_train, y_test,"binary")
x_test_rf_pred = x_test_original
x_test_rf_pred["label"] = y_test
x_test_rf_pred["prediction"] = pred
x_test_rf_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_random_forest_prediction.csv")


--------Random Forest-----------


In [95]:
pred = xgboost(x_train, x_test, y_train, y_test, "binary")
x_test_xg_pred = x_test_original
x_test_xg_pred["label"] = y_test
x_test_xg_pred["prediction"] = pred
x_test_xg_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_xgboost_prediction.csv")


In [96]:
pred = adaboost(x_train, x_test, y_train, y_test, "binary")
x_test_ada_pred = x_test_original
x_test_ada_pred["label"] = y_test
x_test_ada_pred["prediction"] = pred
x_test_ada_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_adaboost_prediction.csv")


In [97]:
pred = svm("linear", x_train, x_test, y_train, y_test, "binary")
x_test_svm_pred = x_test_original
x_test_svm_pred["label"] = y_test
x_test_svm_pred["prediction"] = pred
x_test_svm_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_svm_prediction.csv")

--------Support Vector Classifier linear-----------


In [98]:
pred = svm("rbf", x_train, x_test, y_train, y_test, "binary")
x_test_svmrbf_pred = x_test_original
x_test_svmrbf_pred["label"] = y_test
x_test_svmrbf_pred["prediction"] = pred
x_test_svmrbf_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_svm_rbf_prediction.csv")

--------Support Vector Classifier rbf-----------


In [99]:
pred = svm("poly", x_train, x_test, y_train, y_test, "binary")
x_test_svmpoly_pred = x_test_original
x_test_svmpoly_pred["label"] = y_test
x_test_svmpoly_pred["prediction"] = pred
x_test_svmpoly_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_svm_poly_prediction.csv")


--------Support Vector Classifier poly-----------


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
